In [ ]:
%matplotlib inline
# import matplotlib.image as mpimg
# from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from tf_aerial_images import *
from sklearn import linear_model
from types import SimpleNamespace 
%load_ext autoreload
%autoreload 2

In [ ]:
# remind of confusion matrix
print('Confusion matrix:')
confmat = pd.DataFrame(
    data = [['True Negative (TN)', 'False Negative (FN)'], ['False Positive (FP)', 'True Positive (TP)']],
    index = pd.MultiIndex(
        levels=[['predicted (Z)'], ['0', '1']],
        labels=[[0, 0], [0, 1]]),
    columns = pd.MultiIndex(
        levels=[['actual (Y)'], ['0 (background)', '1 (road)']],
        labels=[[0, 0], [0, 1]]),
)
display(confmat)
print('TN + FN + FP + TP = number of predictions')

## 1. Load data

In [ ]:
# Loaded a set of images
n = 20

imgs, gt_imgs = load_images()#n)

imgs[0].shape, gt_imgs[0].shape

In [ ]:
# # Show first image and its groundtruth image
# cimg = concatenate_images(imgs[1], gt_imgs[1])
# fig1 = plt.figure(figsize=(20, 10))
# plt.imshow(cimg, cmap='Greys_r', vmin=0, vmax=1)

## 2. Generate the input and extract the outputs.
From each image extract the inputs (patches) and generate a list of features for each one of them.

In [ ]:
t = 99 # number of images to use fore the training
train = SimpleNamespace()
train.X = imgs_to_inputs(imgs[:t])
train.Y = imgs_to_outputs(gt_imgs[:t])
train.indices = range(t) # indices on the trained images

test = SimpleNamespace()
test.X = imgs_to_inputs(imgs[t:])
test.Y = imgs_to_outputs(gt_imgs[t:])
test.indices = range(t, len(imgs))

train.X.shape, train.Y.shape

Just check the **output** has been generated correctly.

In [ ]:
# Print feature statistics
Y0 = np.where(train.Y==0)[0]
Y1 = np.where(train.Y==1)[0]
print(str(len(Y0)) + ' inputs are classified as background (0)')
print(str(len(Y1)) + ' inputs are classified as road (1)')

## 3. Train a model

- Logistic regression:

In [ ]:
logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced")
# train
logreg.fit(train.X, train.Y)
# predict
train.Z = logreg.predict(train.X)
train.Z.shape

In [ ]:
# display some stats
stats(train.Z, train.Y)

## 4. Validation

In [ ]:
test.Z = logreg.predict(test.X)

In [ ]:
stats(test.Z, test.Y)

In [ ]:
i = 0
image = imgs[test.indices[i]]
# lenght of the prediction of one image
len_pred = int(len(test.Z)/len(test.indices))
prediction = test.Z[len_pred*i : len_pred*(i+1)]
display_prediction(image, prediction)

### idea 1 


In [ ]:
X_list, Y = initialize_input_output(imgs, gt_imgs)

In [ ]:
X = np.array(flatten([flatten(matr) for matr in X_list]))
output_patches = flatten([flatten(img_crop_matr(img)) for img in gt_imgs])
Y = np.array([int(value_to_class(np.mean(patch))) for patch in output_patches])
X.shape, Y.shape

In [ ]:
models = train_models(imgs, gt_imgs, n_layers=4)

In [ ]:
i = 40
Z, Y = test_models(imgs[i:i+1], gt_imgs[i:i+1], models=models)
Z.shape, Y.shape

In [ ]:
# i = 5
image = imgs[i]
# lenght of the prediction of one image
# len_pred = int(len(Z)/2)
# prediction = Z[len_pred*i : len_pred*(i+1)]
prediction = Z
display_prediction(image, prediction)